In [27]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import itertools
from PIL import Image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix

In [28]:
# Empty dataframe
data = pd.DataFrame(columns=['image_path', 'label'])

labels = {
    r"C:\Users\priya\OneDrive\Desktop\greenAI\day-10\new\data\water" : "Water",
    r"C:\Users\priya\OneDrive\Desktop\greenAI\day-10\new\data\cloudy" : "Cloudy",
    r"C:\Users\priya\OneDrive\Desktop\greenAI\day-10\new\data\green_area" : "Green",
    r"C:\Users\priya\OneDrive\Desktop\greenAI\day-10\new\data\desert" : "Desert",
}

In [29]:
#validate the folder path
rows = []
for folder in labels:
    if not os.path.exists(folder):
        print(f"Folder {folder} does not exist. Please check the path.")
        continue
    for image_name in os.listdir(folder):
        image_path = os.path.join(folder, image_name)
        if os.path.isfile(image_path):
            rows.append({'image_path': image_path, 'label': labels[folder]})

data = pd.DataFrame(rows)
print(data)

                                             image_path   label
0     C:\Users\priya\OneDrive\Desktop\greenAI\day-10...   Water
1     C:\Users\priya\OneDrive\Desktop\greenAI\day-10...   Water
2     C:\Users\priya\OneDrive\Desktop\greenAI\day-10...   Water
3     C:\Users\priya\OneDrive\Desktop\greenAI\day-10...   Water
4     C:\Users\priya\OneDrive\Desktop\greenAI\day-10...   Water
...                                                 ...     ...
5626  C:\Users\priya\OneDrive\Desktop\greenAI\day-10...  Desert
5627  C:\Users\priya\OneDrive\Desktop\greenAI\day-10...  Desert
5628  C:\Users\priya\OneDrive\Desktop\greenAI\day-10...  Desert
5629  C:\Users\priya\OneDrive\Desktop\greenAI\day-10...  Desert
5630  C:\Users\priya\OneDrive\Desktop\greenAI\day-10...  Desert

[5631 rows x 2 columns]


In [30]:
data.to_csv("image_dataset.csv" , index=False)

In [31]:
df = pd.read_csv("image_dataset.csv")
df

,image_path,label
0,C:\Users\priya\OneDrive\Desktop\greenAI\day-10...,Water
1,C:\Users\priya\OneDrive\Desktop\greenAI\day-10...,Water
2,C:\Users\priya\OneDrive\Desktop\greenAI\day-10...,Water
3,C:\Users\priya\OneDrive\Desktop\greenAI\day-10...,Water
4,C:\Users\priya\OneDrive\Desktop\greenAI\day-10...,Water
...,...,...
5626,C:\Users\priya\OneDrive\Desktop\greenAI\day-10...,Desert
5627,C:\Users\priya\OneDrive\Desktop\greenAI\day-10...,Desert
5628,C:\Users\priya\OneDrive\Desktop\greenAI\day-10...,Desert
5629,C:\Users\priya\OneDrive\Desktop\greenAI\day-10...,Desert


In [32]:
train_df, test_df = train_test_split(data, test_size=0.2, random_state=42, stratify=data['label'])

In [33]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    vertical_flip=True,
    rotation_range=20,
    fill_mode='nearest',
)

In [34]:
test_datagen = ImageDataGenerator(rescale=1./255)

In [35]:
train_generator = train_datagen.flow_from_dataframe(
    dataframe=train_df,
    directory=None,
    x_col='image_path',
    y_col='label',
    target_size=(255, 255),
    batch_size=32,
    class_mode='categorical',
    shuffle=True
)

Found 4504 validated image filenames belonging to 4 classes.


In [36]:
train_generator = train_datagen.flow_from_dataframe(
    dataframe=train_df,
    directory=None,
    x_col='image_path',
    y_col='label',
    target_size=(255, 255),
    batch_size=32,
    class_mode='categorical',
    shuffle=True
)

Found 4504 validated image filenames belonging to 4 classes.


In [37]:
test_generator = test_datagen.flow_from_dataframe(
    dataframe=test_df,
    directory=None,
    x_col='image_path',
    y_col='label',
    target_size=(255, 255),
    batch_size=32,
    class_mode='categorical',
    shuffle=False
)

Found 1127 validated image filenames belonging to 4 classes.


In [39]:
# Get the number of classes from the training generator
num_classes = len(train_generator.class_indices)

In [40]:
cnnmodel = Sequential()
cnnmodel.add(Conv2D(32, (3, 3), input_shape=(255, 255, 3), activation='relu'))
cnnmodel.add(MaxPooling2D(pool_size=(2, 2)))
cnnmodel.add(Conv2D(64, (3, 3), activation='relu'))
cnnmodel.add(MaxPooling2D(pool_size=(2, 2)))
cnnmodel.add(Conv2D(128, (3, 3), activation='relu'))
cnnmodel.add(MaxPooling2D(pool_size=(2, 2)))
cnnmodel.add(Flatten())
cnnmodel.add(Dense(128, activation='relu'))
cnnmodel.add(Dense(num_classes, activation='softmax'))  # Output layer for classification
cnnmodel.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
cnnmodel.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_3 (Conv2D)           (None, 253, 253, 32)      896       
                                                                 
 max_pooling2d_3 (MaxPoolin  (None, 126, 126, 32)      0         
 g2D)                                                            
                                                                 
 conv2d_4 (Conv2D)           (None, 124, 124, 64)      18496     
                                                                 
 max_pooling2d_4 (MaxPoolin  (None, 62, 62, 64)        0         
 g2D)                                                            
                                                                 
 conv2d_5 (Conv2D)           (None, 60, 60, 128)       73856     
                                                                 
 max_pooling2d_5 (MaxPoolin  (None, 30, 30, 128)      

In [4]:
import tensorflow as tf

tf.debugging.set_log_device_placement(True)

In [5]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))
print("TensorFlow is using:", "GPU" if tf.config.list_physical_devices('GPU') else "CPU")

Num GPUs Available:  0
TensorFlow is using: CPU


In [47]:
history = cnnmodel.fit(
    train_generator,
    epochs=10,
    validation_data=test_generator,
)

Epoch 1/10
141/141 [==============================] - 552s 4s/step - loss: 0.4753 - accuracy: 0.7782 - val_loss: 0.6824 - val_accuracy: 0.7169
Epoch 2/10
141/141 [==============================] - 402s 3s/step - loss: 0.3827 - accuracy: 0.8393 - val_loss: 0.3649 - val_accuracy: 0.8589
Epoch 3/10
115/141 [=======================>......] - ETA: 1:15 - loss: 0.3588 - accuracy: 0.8552

KeyboardInterrupt: 

# Enabling and Verifying GPU Usage in TensorFlow/Keras

To use your GPU for deep learning in this notebook, follow these steps:

**1. Check your hardware:**
- You need an NVIDIA GPU (GeForce, Quadro, or Tesla series).

**2. Install NVIDIA GPU drivers:**
- Download and install the latest drivers for your GPU from the [NVIDIA website](https://www.nvidia.com/Download/index.aspx).

**3. Install CUDA Toolkit and cuDNN:**
- Download the CUDA Toolkit version compatible with your TensorFlow version from the [CUDA Toolkit Archive](https://developer.nvidia.com/cuda-toolkit-archive).
- Download cuDNN for the same CUDA version from the [NVIDIA cuDNN page](https://developer.nvidia.com/cudnn).
- Follow the installation instructions for both.

**4. Install TensorFlow with GPU support:**
- In your terminal or Anaconda prompt, run:
  ```
  pip install tensorflow
  ```
- The latest TensorFlow package includes GPU support if compatible drivers and libraries are present.

**5. Verify GPU is detected:**
- Add and run the following cell in your notebook:
  ```python
  import tensorflow as tf
  print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))
  print("TensorFlow is using:", "GPU" if tf.config.list_physical_devices('GPU') else "CPU")
  ```
- If you see a GPU listed, TensorFlow will use it automatically for training.

**Troubleshooting:**
- If no GPU is detected, check your driver, CUDA, and cuDNN versions for compatibility with your TensorFlow version.
- You can find compatibility tables in the [TensorFlow documentation](https://www.tensorflow.org/install/source#gpu).


In [6]:
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  0


In [7]:
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  0
